In [1]:
import pandas as pd
from pymongo import MongoClient
import duckdb
from oaklib import get_adapter

In [2]:
# For the BBOP/NMDC MongoDB containing NCBI metadata

MONGO_USERNAME = None
MONGO_PASSWORD = None
MONGO_HOST = "localhost"
MONGO_PORT = 27017
MONGO_DATABASE = "ncbi_metadata"
BIOPROJECTS_COLLECTION = "bioprojects"
BIOSAMPLES_COLLECTION = "biosamples"
BIOSAMPLES_BIOPROJECTS_COLLECTION = "sra_biosamples_bioprojects"


In [3]:
# "Potential_import_SRA_Jan2024"
SHEET_ID = "1432d4WGdO5aSU2SvHBSvlqWmJ1NiEOTJ4UGQq1zR_ho"

In [4]:
NCBI_BIOSAMPLES_DUCKDB_FILE = "../../../local/ncbi_biosamples.duckdb"

In [5]:
CSV_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv"

In [6]:
envo_adapter_string = 'sqlite:obo:envo'

In [7]:
output_tsv = "simons_wishlist_env_triads.tsv"

In [8]:
# Read the CSV into a pandas DataFrame
df = pd.read_csv(CSV_URL)


In [9]:
# Display the first few rows
df

,"## Bioproject selected based on: ## At least 50 metagenomes (10M reads +), not currently in IMG (as far as I know), environmental / terrestrial / soil / plant ## For IMG imports, looked at cases where MAGs are not already available in NCBI, and potential for interesting new diversity ## For NMDC imports, looked more specifically at soil and related, and nice metadata BioProject Id",# metaG 10M+,Project name,Environment(s),Publication,MAGs in NCBI,Import to IMG ?,Import to NMDC ?,Notes
0,PRJEB52368,827,Metagenomic sequencing of Tara Pacific coral s...,Marine/coral,NaN,NaN,No,No,"Tara, unpublished, probably stay away"
1,PRJNA352737,721,HOT ALOHA metagenomic time and depth series,Marine,10.1038/s41564-017-0008-3,Yes (197),No,No,"Already has MAGs in NCBI, and oceanic so not h..."
2,PRJNA385736,688,Marine amplicons/metagenomes from Australian ...,Marine,NaN,NaN,Yes,Maybe ?,"Some paired metaG and metaT, but marine and no..."
3,PRJNA656268,572,Bio-GO-SHIP: Global marine 'omics studies of r...,Marine,No but NSF grant,Maybe (see TPA),No,No,Has a TPA and already in MGnify. 10.1093/nar/g...
4,PRJNA385854,470,Marine metagenomes from the bioGEOTRACES project,Marine,10.1038/sdata.2018.176,NaN,Yes,Maybe ?,"Rich metadata, could be good if no MAGs (Data ..."
...,...,...,...,...,...,...,...,...,...
108,PRJEB35627,50,Seasonal and diel patterns of bacteriophages i...,Marine,NaN,NaN,No,No,NaN
109,PRJNA476799,50,Greenhouse Vegetable Surfaces Raw sequence reads,Plant,NaN,NaN,No,No,"Food related, not high priority"
110,PRJNA691683,50,Topsoil viromes from five types of land uses,Soil,NaN,NaN,No,No,More for IMG/VR ?
111,PRJNA798446,50,Shotgun metagenome of microbial community in m...,Soil,NaN,NaN,No,No,no metadata


In [10]:
# Get the leftmost column (first column)
bioproj_accession_column = df.iloc[:, 0]  # Select the first column

# Get unique values
bioproj_accession_values = list(bioproj_accession_column.unique())
bioproj_accession_values.sort()

In [11]:
# Display the unique values
bioproj_accession_values

['PRJEB18675',
 'PRJEB27870',
 'PRJEB31530',
 'PRJEB34634',
 'PRJEB34883',
 'PRJEB35627',
 'PRJEB35640',
 'PRJEB35770',
 'PRJEB38290',
 'PRJEB38681',
 'PRJEB40760',
 'PRJEB41174',
 'PRJEB41834',
 'PRJEB4352',
 'PRJEB44163',
 'PRJEB44309',
 'PRJEB44414',
 'PRJEB45634',
 'PRJEB52368',
 'PRJEB52406',
 'PRJEB52452',
 'PRJEB52753',
 'PRJEB55522',
 'PRJEB62460',
 'PRJEB66294',
 'PRJEB9691',
 'PRJEB9742',
 'PRJNA1000042',
 'PRJNA1000596',
 'PRJNA1001993',
 'PRJNA1002763',
 'PRJNA1014704',
 'PRJNA1028263',
 'PRJNA1035420',
 'PRJNA1035643',
 'PRJNA308326',
 'PRJNA329908',
 'PRJNA352737',
 'PRJNA358725',
 'PRJNA379303',
 'PRJNA385736',
 'PRJNA385854',
 'PRJNA385855',
 'PRJNA386568',
 'PRJNA389803',
 'PRJNA395437',
 'PRJNA429141',
 'PRJNA448773',
 'PRJNA449266',
 'PRJNA450295',
 'PRJNA450643',
 'PRJNA473136',
 'PRJNA476799',
 'PRJNA528368',
 'PRJNA530708',
 'PRJNA545144',
 'PRJNA558772',
 'PRJNA577476',
 'PRJNA588686',
 'PRJNA608274',
 'PRJNA628860',
 'PRJNA629394',
 'PRJNA632564',
 'PRJNA640378'

In [12]:
len(bioproj_accession_values)

113

In [13]:
if MONGO_USERNAME is not None and MONGO_PASSWORD is not None:
    username = MONGO_USERNAME
    password = MONGO_PASSWORD
    host = MONGO_HOST
    port = MONGO_PORT

    # Build the connection string with authentication.
    connection_string = f"mongodb://{username}:{password}@{host}:{port}"
else:
    # Default connection to unauthenticated MongoDB.
    host = MONGO_HOST
    port = MONGO_PORT
    connection_string = f"mongodb://{host}:{port}"

# Create the client connection.
client = MongoClient(connection_string)

In [14]:
# --------------------------
# Select Database
# --------------------------

db = client[MONGO_DATABASE]  # Dynamically select database


In [15]:
biosamples_bioprojects_collection = db[BIOSAMPLES_BIOPROJECTS_COLLECTION]

In [16]:
# MongoDB query using the `$in` operator
query = {"bioproject_accession": {"$in": bioproj_accession_values}}

# Fetch matching documents
wishlist_biosamples_bioprojects = list(biosamples_bioprojects_collection.find(query))


In [17]:
len(wishlist_biosamples_bioprojects)

30161

In [18]:
# Print the results
for doc in wishlist_biosamples_bioprojects[0:9]:
    print(doc)

{'_id': ObjectId('679b82323b7bd066cbcbb9f2'), 'biosample_accession': 'SAMEA30188668', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f3'), 'biosample_accession': 'SAMEA30189418', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f4'), 'biosample_accession': 'SAMEA30190168', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f5'), 'biosample_accession': 'SAMEA30190918', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f6'), 'biosample_accession': 'SAMEA30191668', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f7'), 'biosample_accession': 'SAMEA30192418', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f8'), 'biosample_accession': 'SAMEA30193168', 'bioproject_accession': 'PRJEB18675'}
{'_id': ObjectId('679b82323b7bd066cbcbb9f9'), 'biosample_accession': 'SAMEA30193918', 'bioproject_accession': 'PRJEB18675'}
{'_id': 

In [19]:
biosamples_collection = db[BIOSAMPLES_COLLECTION]

In [20]:
# Extract the biosample_accession values
biosample_accession_list = [item["biosample_accession"] for item in wishlist_biosamples_bioprojects]

# MongoDB query using `$in`
query = {"accession": {"$in": biosample_accession_list}}

# Fetch matching documents
wishlist_biosamples = list(biosamples_collection.find(query))

In [21]:
len(wishlist_biosamples)

30161

In [22]:
wishlist_biosamples[0]

{'_id': ObjectId('677f66eb0a6241ac79325d55'),
 'access': 'public',
 'publication_date': '2022-06-06T00:00:00.000',
 'last_update': '2024-06-26T10:36:46.000',
 'submission_date': '2022-09-23T14:20:12.056',
 'id': '30986217',
 'accession': 'SAMEA110022181',
 'Ids': {'Id': [{'content': 'SAMEA110022181',
    'db': 'BioSample',
    'is_primary': '1'},
   {'content': 'ERS11966019', 'db': 'SRA'}]},
 'Description': {'Title': {'content': '2'},
  'Organism': {'taxonomy_id': '256318',
   'taxonomy_name': 'metagenome',
   'OrganismName': {'content': 'metagenome'}}},
 'Owner': {'Name': {'content': 'EBI'}},
 'Models': {'Model': {'content': 'Generic'}},
 'Package': {'content': 'Generic.1.0', 'display_name': 'Generic'},
 'Attributes': {'Attribute': [{'content': 'GR',
    'attribute_name': 'Country',
    'harmonized_name': 'geo_loc_name',
    'display_name': 'geographic location'},
   {'content': 'ERC000022', 'attribute_name': 'ENA-CHECKLIST'},
   {'content': 'CEH', 'attribute_name': 'INSDC center name

In [23]:
# Convert to dictionary for O(1) lookup
bioproject_dict = {item["biosample_accession"]: item["bioproject_accession"] for item in wishlist_biosamples_bioprojects}


In [24]:
# List of relevant harmonized names
target_harmonized_names = {"env_broad_scale", "env_local_scale", "env_medium"}

data = []
for doc in wishlist_biosamples:
    biosample_accession = doc.get("accession")
    bioproject_accession = bioproject_dict.get(biosample_accession)

    # Get attributes and ensure it's always a list
    attributes = doc.get("Attributes", {}).get("Attribute", [])
    if isinstance(attributes, dict):
        attributes = [attributes]

    # Collect attributes into a dictionary
    attr_dict = {name: None for name in target_harmonized_names}  # Initialize with None
    for attr in attributes:
        harmonized_name = attr.get("harmonized_name")
        if harmonized_name in target_harmonized_names:
            attr_dict[harmonized_name] = attr.get("content")

    # Append the structured row
    data.append({
        "bioproject_accession": bioproject_accession,
        "biosample_accession": biosample_accession,
        "env_broad_scale": attr_dict["env_broad_scale"],
        "env_local_scale": attr_dict["env_local_scale"],
        "env_medium": attr_dict["env_medium"],
    })


# Convert to pandas DataFrame
df = pd.DataFrame(data)

In [25]:
df

,bioproject_accession,biosample_accession,env_broad_scale,env_local_scale,env_medium
0,PRJEB52753,SAMEA110022181,terrestrial biome,None,soil
1,PRJEB52753,SAMEA110027444,terrestrial biome,None,soil
2,PRJEB52452,SAMEA110646458,Trades Biome,None,"particulate matter (ENVO:01000060), including ..."
3,PRJEB52452,SAMEA110646459,Trades Biome,None,"particulate matter (ENVO:01000060), including ..."
4,PRJEB52452,SAMEA110646460,Trades Biome,None,"particulate matter (ENVO:01000060), including ..."
...,...,...,...,...,...
30156,PRJNA1035643,SAMN38096308,xeric shrubland biome,terrestrial environmental zone,surface soil
30157,PRJNA1035643,SAMN38096309,xeric shrubland biome,terrestrial environmental zone,surface soil
30158,PRJNA1035643,SAMN38096310,xeric shrubland biome,terrestrial environmental zone,surface soil
30159,PRJNA1035643,SAMN38096311,xeric shrubland biome,terrestrial environmental zone,surface soil


In [26]:
df.columns

Index(['bioproject_accession', 'biosample_accession', 'env_broad_scale',
       'env_local_scale', 'env_medium'],
      dtype='object')

In [27]:
duck_conn = duckdb.connect(NCBI_BIOSAMPLES_DUCKDB_FILE)

In [28]:
# Convert biosample_accession_list to a Pandas DataFrame
accession_df = pd.DataFrame(biosample_accession_list, columns=["accession"])


In [29]:
# Register the DataFrame as a temporary DuckDB table (in-memory, no writes to disk)
duck_conn.register("accession_filter", accession_df)

In [30]:
# Define the query using the registered table
query = """
WITH biosample_subset AS (
    SELECT b.id, b.accession
    FROM main.biosample b
    JOIN accession_filter af
        ON b.accession = af.accession
),
asserted_data AS (
    SELECT
        b.id,
        b.accession,
        ctns.harmonized_name,
        ca.curie,
        'asserted' AS source
    FROM
        main.curies_asserted ca
    JOIN main.contexts_to_normalized_strings ctns
        ON ca.id = ctns.normalized_context_string_id
    JOIN biosample_subset b
        ON ctns.id = b.id
),
ner_data AS (
    SELECT
        b.id,
        b.accession,
        ctns.harmonized_name,
        cn.curie,
        cn.coverage_sum,
        'NER' AS source
    FROM
        main.curies_ner cn
    JOIN main.contexts_to_normalized_strings ctns
        ON cn.id = ctns.normalized_context_string_id
    JOIN biosample_subset b
        ON ctns.id = b.id
    WHERE
        cn.is_longest_match = TRUE
        AND cn.subsumed = FALSE
),
ranked_ner AS (
    SELECT
        fd.id,
        fd.accession,
        fd.harmonized_name,
        fd.curie,
        fd.source,
        ROW_NUMBER() OVER (PARTITION BY fd.id, fd.harmonized_name ORDER BY fd.coverage_sum DESC) AS rank
    FROM
        ner_data fd
)
SELECT id, accession, harmonized_name, curie, source
FROM asserted_data
UNION ALL
SELECT id, accession, harmonized_name, curie, source
FROM ranked_ner
WHERE rank = 1
ORDER BY id, harmonized_name, source;
"""


In [31]:
# Execute query and fetch results
id_harmonized_name_best_curies = duck_conn.execute(query).fetchdf()

In [32]:
id_harmonized_name_best_curies

,id,accession,harmonized_name,curie,source
0,2872651,SAMEA2591063,env_broad_scale,ENVO:00000447,NER
1,2872651,SAMEA2591063,env_broad_scale,ENVO:00000447,asserted
2,2872651,SAMEA2591063,env_local_scale,ENVO:00002042,NER
3,2872651,SAMEA2591063,env_local_scale,ENVO:00002042,asserted
4,2872651,SAMEA2591063,env_medium,ENVO:01000060,NER
...,...,...,...,...,...
41248,38096312,SAMN38096312,env_broad_scale,ENVO:01000218,NER
41249,38096312,SAMN38096312,env_local_scale,ENVO:01001199,NER
41250,38096312,SAMN38096312,env_medium,ENVO:02000059,NER
41251,42755192,SAMEA115861732,env_broad_scale,ENVO:00000873,NER


In [33]:
envo_adapter = get_adapter(envo_adapter_string)

In [34]:
# Extract unique EnvO CURIEs
all_mined_curies = id_harmonized_name_best_curies["curie"].unique()


In [35]:
# Fetch labels for all EnvO classes
all_mined_curie_labels = {}
for curie in all_mined_curies:
    label = envo_adapter.label(curie)
    if label:
        all_mined_curie_labels[curie] = label


In [36]:
id_harmonized_name_best_curies["curie_label"] = id_harmonized_name_best_curies["curie"].map(
    lambda curie: f"{all_mined_curie_labels.get(curie, 'Unknown')} [{curie}]"
)

In [37]:
# Map CURIEs to labels, defaulting to the CURIE itself if the label is missing
id_harmonized_name_best_curies

,id,accession,harmonized_name,curie,source,curie_label
0,2872651,SAMEA2591063,env_broad_scale,ENVO:00000447,NER,marine biome [ENVO:00000447]
1,2872651,SAMEA2591063,env_broad_scale,ENVO:00000447,asserted,marine biome [ENVO:00000447]
2,2872651,SAMEA2591063,env_local_scale,ENVO:00002042,NER,surface water [ENVO:00002042]
3,2872651,SAMEA2591063,env_local_scale,ENVO:00002042,asserted,surface water [ENVO:00002042]
4,2872651,SAMEA2591063,env_medium,ENVO:01000060,NER,particulate environmental material [ENVO:01000...
...,...,...,...,...,...,...
41248,38096312,SAMN38096312,env_broad_scale,ENVO:01000218,NER,xeric shrubland biome [ENVO:01000218]
41249,38096312,SAMN38096312,env_local_scale,ENVO:01001199,NER,terrestrial environmental zone [ENVO:01001199]
41250,38096312,SAMN38096312,env_medium,ENVO:02000059,NER,surface soil [ENVO:02000059]
41251,42755192,SAMEA115861732,env_broad_scale,ENVO:00000873,NER,freshwater biome [ENVO:00000873]


In [38]:

# Pivot so that each harmonized_name becomes a separate column
biosample_curie_pivot = (
    id_harmonized_name_best_curies
    .groupby(["id", "accession", "harmonized_name"])["curie_label"]
    .apply(lambda x: "; ".join(sorted(set(x))))  # Aggregate CURIe/label pairs
    .unstack(fill_value="")  # Pivot harmonized_name columns
    .reset_index()
)


In [39]:
# Ensure the correct column order, filling missing columns if needed
expected_columns = ["id", "accession", "env_broad_scale", "env_local_scale", "env_medium"]
for col in expected_columns:
    if col not in biosample_curie_pivot.columns:
        biosample_curie_pivot[col] = ""  # Add missing harmonized_name columns


In [40]:
biosample_curie_pivot = biosample_curie_pivot[expected_columns]  # Reorder columns


In [41]:
# Reset column names to fix any unintended MultiIndex issues
biosample_curie_pivot.columns.name = None  # Remove any lingering index name
biosample_curie_pivot = biosample_curie_pivot.rename_axis(None, axis=1)  # Remove any index label

In [42]:
biosample_curie_pivot

,id,accession,env_broad_scale,env_local_scale,env_medium
0,2872651,SAMEA2591063,marine biome [ENVO:00000447],surface water [ENVO:00002042],particulate environmental material [ENVO:01000...
1,2872652,SAMEA2611380,marine biome [ENVO:00000447],surface water [ENVO:00002042],particulate environmental material [ENVO:01000...
2,2872653,SAMEA2591077,marine biome [ENVO:00000447],deep chlorophyll maximum layer [ENVO:01000326],particulate environmental material [ENVO:01000...
3,2872654,SAMEA2591082,marine biome [ENVO:00000447],deep chlorophyll maximum layer [ENVO:01000326],particulate environmental material [ENVO:01000...
4,2872655,SAMEA2611378,marine biome [ENVO:00000447],surface water [ENVO:00002042],particulate environmental material [ENVO:01000...
...,...,...,...,...,...
14681,38096309,SAMN38096309,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]
14682,38096310,SAMN38096310,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]
14683,38096311,SAMN38096311,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]
14684,38096312,SAMN38096312,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]


In [43]:
# Perform a full outer join on biosample_accession (df) and accession (biosample_curie_pivot)
merged_df = df.merge(
    biosample_curie_pivot,
    left_on="biosample_accession",
    right_on="accession",
    how="outer",  # Ensures all rows from both DataFrames are kept
    suffixes=("_asserted", "_mined")  # Suffix for column disambiguation
)


In [44]:
# Drop the duplicate "accession" column after merging (since it matches "biosample_accession")
merged_df = merged_df.drop(columns=["accession"])

In [45]:
# Rename "id" to "biosample_id"
merged_df = merged_df.rename(columns={"id": "biosample_id"})

In [46]:
# Convert "biosample_id" to integer, handling potential NaNs (replace NaNs with -1 and convert)
merged_df["biosample_id"] = merged_df["biosample_id"].fillna(-1).astype(int)

In [47]:
# Reorder columns: Move "biosample_id" next to "biosample_accession"
column_order = (
    ["bioproject_accession", "biosample_id", "biosample_accession"] +
    [col for col in merged_df.columns if col not in ["bioproject_accession", "biosample_id", "biosample_accession"]]
)

merged_df = merged_df[column_order]

In [48]:
merged_df

,bioproject_accession,biosample_id,biosample_accession,env_broad_scale_asserted,env_local_scale_asserted,env_medium_asserted,env_broad_scale_mined,env_local_scale_mined,env_medium_mined
0,PRJEB52753,30986217,SAMEA110022181,terrestrial biome,None,soil,terrestrial biome [ENVO:00000446],,soil [ENVO:00001998]
1,PRJEB52753,30986218,SAMEA110027444,terrestrial biome,None,soil,terrestrial biome [ENVO:00000446],,soil [ENVO:00001998]
2,PRJEB52452,33205310,SAMEA110646458,Trades Biome,None,"particulate matter (ENVO:01000060), including ...",biome [ENVO:00000428],,particulate environmental material [ENVO:01000...
3,PRJEB52452,33205311,SAMEA110646459,Trades Biome,None,"particulate matter (ENVO:01000060), including ...",biome [ENVO:00000428],,particulate environmental material [ENVO:01000...
4,PRJEB52452,33205312,SAMEA110646460,Trades Biome,None,"particulate matter (ENVO:01000060), including ...",biome [ENVO:00000428],,particulate environmental material [ENVO:01000...
...,...,...,...,...,...,...,...,...,...
30156,PRJNA1035643,38096308,SAMN38096308,xeric shrubland biome,terrestrial environmental zone,surface soil,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]
30157,PRJNA1035643,38096309,SAMN38096309,xeric shrubland biome,terrestrial environmental zone,surface soil,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]
30158,PRJNA1035643,38096310,SAMN38096310,xeric shrubland biome,terrestrial environmental zone,surface soil,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]
30159,PRJNA1035643,38096311,SAMN38096311,xeric shrubland biome,terrestrial environmental zone,surface soil,xeric shrubland biome [ENVO:01000218],terrestrial environmental zone [ENVO:01001199],surface soil [ENVO:02000059]


In [49]:
merged_df.to_csv(output_tsv, index=False, sep="\t")

In [50]:
# close duck_conn
duck_conn.close()